In [5]:
folders = [
    '/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_females',
    '/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_males',
    '/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Females',
    '/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males',
    '/n/groups/datta/min/dominance_v1',
    '/n/groups/datta/min/community_v1',
    # '/n/groups/datta/min/wheel_062023',
    # '/n/groups/datta/min/cas_behavior_01',
    # '/n/groups/datta/min/sham_behavior_01',
]

In [2]:
# old model trained on females
# model_path = '/n/groups/datta/win/longtogeny/size_norm/models/param_scan/bc632741-9de6-44bf-8b3c-e10e835948f8/Autoencoder-epoch=73-val_loss=5.84e-04.ckpt'

# new model trained on males
model_path = '/n/groups/datta/win/longtogeny/size_norm/models/pre_final_model/model.pt'

In [3]:
script = '''#!/bin/env bash
#SBATCH -c 1
#SBATCH -n 1
#SBATCH --mem=8G
#SBATCH -p gpu_quad
#SBATCH --gres=gpu:1
#SBATCH -t 04:00:00
#SBATCH --output=/n/scratch3/users/w/wg41/tmp/win-size-norm-%j.out

source $HOME/.bashrc
conda activate aging
module load gcc/9.2.0
module load cuda/11.7
python $HOME/code/ontogeny/scripts/02-apply-dnn.py "{data_path}" "{model_path}" --key win_size_norm_frames_v2
'''

In [6]:
for folder in folders:
    new_script = script.format(data_path=folder, model_path=model_path)
    with open('tmp.sh', 'w') as f:
        f.write(new_script)
    !sbatch tmp.sh
!rm tmp.sh

sbatch: Setting QOS to gpuquad_qos
Submitted batch job 11770455
sbatch: Setting QOS to gpuquad_qos
Submitted batch job 11770456
sbatch: Setting QOS to gpuquad_qos
Submitted batch job 11770457
sbatch: Setting QOS to gpuquad_qos
Submitted batch job 11770459
